## FINAL PROYECT 
Guillermo Ruiz , Macarena Vargas, Cristina Hernández, Beatriz Jiménez 


This is out code to model the optimization problem that optimizes the use of a venue of a local community


Libraries

In [1]:
import pyomo.environ as pe
import pyomo.opt as po
model = pe.ConcreteModel()

SETS : 
- $d$: set of days {Monday, Tuesday, Wednesday, Thursday, Friday, Saturday,       Sunday} 
- $a$: set of activities that are scheduled {Pilates, Fitness , Movies}
- $p$: time periods in a day {Morning, Afternoon} 
- $s$ : supervisors {Supervisor1, Supervisor2, ...}
- $Hm$: Hours of the morning (time slots) {1,2,3,4,5}  
- $Ha$: Hours of the afternoon (time slots) {1,2,3,4,5} 

In [ ]:
days_week= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
period = ['Morning', 'Afternoon']
supervisors = ['S1', 'S2', 'S3']
activities = ['Pilates', 'Fitness', 'Movies']
morning_hours = [1, 2, 3, 4, 5]
afternoon_hours = [1, 2, 3, 4, 5]
model.d = pe.Set(initialize = days_week, ordered = True)
model.p = pe.Set(initialize = period, ordered = True)
model.s = pe.Set(initialize = supervisors, ordered = False )
model.a = pe.Set(initialize = activities, ordered = False)
model.hm = pe.Set(initialize= morning_hours, ordered = True)
model.ha = pe.Set(initialize= afternoon_hours, ordered = True)
model.hp = pe.Set(initialize= , ordered = True)


'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.set.FiniteScalarSet'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.set.AbstractFiniteScalarSet'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.set.FiniteScalarSet'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.set.AbstractFiniteScalarSet'>). This is usually
indicative of a modelling error. To avoid thi

PARAMETERS: 
- $C$ : Maximum capacity of the room (max num of people allowed at any time)
- $Demand_{a, d, h}$: Expected number of residents who would like to attend activity a if it starts on day d at time slot h
- $Disponibility_{d, h}$: The room is available at the day d, time slot h {0,1} 
- $MinSes_a$ : Minimum number of weekly sessions of activity a.
- $MaxSes_a$ : Maximum number of weekly sessions of activity a.
- $Duration_a$ : Duration of activity a , in number of consecutive time slots

VARIABLES: 
- $x_{a,d, p, h}$: The activity a starts on day d at period p time h {1,0}
- $n_{t,m}$: Number of attendees of activity a that starts on day d at period d at time slot h
- $y_{a, d, h, s}$: A supervisor s is assigned to activity a at the day d, period p and period d {0, 1}

In [ ]:

model.x = pe.Var(model.a, model.d, model.p, model.hp, within = pe.Binary)
model.n = pe.Var(model.t, model.m, within = pe.NonNegativeIntegers)
model.y = pe.Var(model.a, model.d, model.hp, model.s, within = pe.Binary)

AttributeError: 'ConcreteModel' object has no attribute 'h'

CONSTRAINTS : 

1. Disponibility rule: Room availability during the whole duration: if activity a starts at (d, h) the room must be available in all slots it occupies. 


 $x_{a,d, h} \leq Disponibility_{d, t}       \forall a , h, t$

 where $a \in A, h \in H, t\in H: t \in [h , h + Duration_{a} -1] $



In [ ]:
def disponibility_rule(m, a, d, h, t):
    # Solo imponemos la restricción si t está dentro del intervalo [h, h + Duration[a] - 1]
    if t < h or t > h + m.Duration[a] - 1:
        return pe.Constraint.Skip
    # Si t está dentro del intervalo, imponemos:
    # x[a,d,h] <= Disponibility[d,t]
    return m.x[a, d, h] <= m.Disponibility[d, t]

model.DisponibilityConstr = pe.Constraint(
    model.A, model.D, model.H, model.H,  # índices: a, d, h, t
    rule=disponibility_rule
)